# Membuat source model xml untuk OpenQuake

Jalankan file ini di Openquake yang sudah diinstal di komputer masing-masing.

atau:

Jalankan file ini di jupyter dengan cara mengubah kernel ke `openquake`. Jika kernel `openquake` belum ada, lakukan langkah-langkah berikut ini:

Openquake dapat diunduh di sini:
https://downloads.openquake.org/pkgs/windows/oq-engine/OpenQuake_Engine_3.11.5-1.exe

Install beberapa tambahan packages di openquake:

(Asumsi lokasi instalasi openquake di sini: `C:\Program Files\Openquake Engine\python3.6\`, 
jika tidak maka ubah lokasi tersebut sesuai dengan komputer masing-masing)

1. Buka Command Prompt
2. Install `jupyter`: `"C:\Program Files\Openquake Engine\python3.6\python.exe" -m pip install jupyter`
3. Daftarkan ke dalam kernel (opsional): `"C:\Program Files\Openquake Engine\python3.6\python.exe" -m ipykernel install --name "openquake" --display-name "openquake"`


Untuk uji coba, jalankan salah satu baris di bawah ini:

`"C:\Program Files\Openquake Engine\python3.6\Scripts\jupyter.exe" notebook`

atau

`"C:\Program Files\Openquake Engine\python3.6\Scripts\jupyter-notebook.exe"`

## 1. Import packages

Hal ini memastikan apakah python yang digunakan sudah menyediakan packages yang dibutuhkan.

In [21]:
from custom_openquake import *

## 2. Patahan

In [54]:
import geojson
import xml.etree.ElementTree as ET
from openquake.hazardlib.mfd.evenly_discretized import EvenlyDiscretizedMFD

geojsonname = './json_output/json_fault_KumeringNorth_others.geojson'
xmlname = './oqfiles/single_ruptures/ssm_1/b_0/single_sec_rup.xml'

with open(geojsonname) as f:
    data = geojson.load(f)
    
tree = ET.parse(xmlname)
root = tree.getroot()
    
mtk_simple_faults = []
for i, ft in enumerate(data['features']):
    geom = ft['geometry']['coordinates']
    prop = ft['properties']
    line = LineOQ([PointOQ(x, y) for x, y, _ in geom])
    mtk_simple_faults.append(mtkSimpleFaultSource(
        identifier=f"{prop['si']}", 
        name=f"{prop['name']}",
        trt="Active Shallow Crust",
        mag_scale_rel="WC1994", 
        rupt_aspect_ratio=1.0, 
        mfd= EvenlyDiscretizedMFD(
            float(root[0][0][i][3].attrib['minMag']), 
            float(root[0][0][i][3].attrib['binWidth']), 
            [float(num) for num in root[0][0][i][3][0].text.split()]
        ),
        rake=prop['rake'],
    ))
    mtk_simple_faults[i].create_geometry(
        line,
        dip = prop['dip'], 
        upper_depth = prop['up_s_d'], 
        lower_depth = prop['lo_s_d'],
        mesh_spacing=1.0
    )
    
model_simple_faults = mtkSourceModel(
    identifier="1001", 
    name="Merge Faults 1", 
    sources=mtk_simple_faults
)
model_simple_faults.serialise_to_nrml('./xmlfiles/merge_faults_1.xml', use_defaults=False)

## \#\#\# Opsional \#\#\#

In [38]:
import xml.etree.ElementTree as ET
from openquake.hazardlib.sourcewriter import write_source_model
from openquake.hazardlib.mfd.evenly_discretized import EvenlyDiscretizedMFD

geod = pyproj.Geod(ellps="WGS84")

xmlname = '/Users/yudhastyawan/Downloads/single_ruptures/ssm_2/b_0/single_sec_rup.xml'

tree = ET.parse(xmlname)
root = tree.getroot()

mtk_simple_faults = []
for i, rt in enumerate(root[0][0]):
#     print(float(rt[0][0][0][0].text.split()[2]))
#     print(float(rt[0][0][0][0].text.split()[5]))
    _,_,dist = geod.inv(float(rt[0][0][0][0].text.split()[0]),
                     float(rt[0][0][0][0].text.split()[1]),
                     float(rt[0][0][0][0].text.split()[3]),
                     float(rt[0][0][0][0].text.split()[4]))
    dip = np.round(np.rad2deg(np.arctan2(float(rt[0][0][0][0].text.split()[5]), dist/1000)))
    geom = [map(float,irt[0][0].text.split()[0:2]) for irt in rt[0]]
    line = LineOQ([PointOQ(x, y) for x, y in geom])
    mtk_simple_faults.append(mtkSimpleFaultSource(
        identifier=rt.attrib['id'], 
        name=rt.attrib['name'],
        trt="Active Shallow Crust",
        mag_scale_rel="WC1994", 
        rupt_aspect_ratio=1.0, 
        mfd= EvenlyDiscretizedMFD(
            float(rt[3].attrib['minMag']), 
            float(rt[3].attrib['binWidth']), 
            [float(num) for num in root[0][0][i][3][0].text.split()]
        ),
        rake=float(rt[4].text),
    ))
    mtk_simple_faults[i].create_geometry(
        line,
        dip = dip, 
        upper_depth = float(rt[0][0][0][0].text.split()[2]), 
        lower_depth = float(rt[0][0][0][0].text.split()[5]),
        mesh_spacing=1.0
    )
    
model_simple_faults = mtkSourceModel(
    identifier="1001", 
    name="Merge Faults 1", 
    sources=mtk_simple_faults
)

source_model = model_simple_faults.convert_to_oqhazardlib(
    PoissonTOM(1.0), 1.0, 2.0, 10.0,
    use_defaults=False)
write_source_model('/Users/yudhastyawan/Downloads/single_ruptures/ssm_2/b_0/single_sec_rup_2.xml', 
                   source_model, 
                   name=model_simple_faults.name)
# model_simple_faults.serialise_to_nrml('/Users/yudhastyawan/Downloads/single_ruptures/ssm_2/b_0/single_sec_rup_2.xml', use_defaults=False)

['/Users/yudhastyawan/Downloads/single_ruptures/ssm_2/b_0/single_sec_rup_2.xml']